**Seminar:** Struktur und Interpretation von Computerprogrammen - Grundlegende Programmierkonzepte anhand der Sprache LISP, Frühjahrsemester 2021  
**Dozenten:** Prof. Dr. Christian Tschudin, Dr. Marcel Lüthi  
**Datum:** 5. September 2021  

# Eine Krivine-Maschine in Scheme  

Luka Obser <luka.obser@unibas.ch>  
Reto Krummenacher <reto.krummenacher@unibas.ch>

### Einleitung

#### Motivation
- curry-howard correspondence -> programs as proofs/proofs as programs, krivine asks "what are meaningful behaviours of
such programs and how can we evaluate them
- lambda calc as smallest formal system (3 rules for construction of terms and, in this case, 2 reduktion rules (alpha and beta reductions))

### 1. Hintergrund und Theorie
Es folgen kompakte und in sich abgeschlossene Definitionen der theoretischen Grundlagen der Krivine-Maschine.


#### 1.a) Lambda-Kalkül Recap
Wir wollen zuerst einen kurzen Auffrischer des Lambda-Kalküls anbieten.
Für eine detailliertere Einführung empfiehlt sich Rojas (2015).

Definition gültiger λ-Terme:
- Variablen: $x$
    - Variablen werden durch Symbole (hier: $a$, $b$, $c$, ...) repräsentiert und stehen für beliebige andernorts
    definierte mathematische Objekte ohne spezifischen Typ. Dies könnten Zahlen oder auch komplexere Konzepte
    wie Funktionen sein. Die Sinngebung und Interpretation ist dem Author der λ-Terme überlassen.
- Abstraktion: $λx.a$
    - Abstraktionen beschreiben das Konzept von Funktionsdefinition, indem sie Variablen (hier $x$) an λ-Terme (hier $a$) binden.
    Das obige Beispiel kann also als $f(x) := a$ interpretiert werden, wobei $x$ hier nicht (mehr) andernorts definiert ist
    (mehr zu den Problemen die daraus resultieren können in **1.b)**).
    - Falls $a$ ebenfalls eine Abstraktion ist, z.B. $λy.b$, entspricht dies Currying. Wir führen deshalb auch die
    syntaktisch gezuckerte Schreibweise $λxy.b$ als Alternative zu $λx.λy.b$ ein.
- Applikation: $a\ b$
    - Applikationen von zwei gültigen λ-Termen $a, b$ entsprechen dem Aufruf von $a$ mit $b$ als Argument.
    - Wir führen wieder syntaktischen Zucker ein, indem wir verschachtelte Applikation wie $(((a\ b)\ c)\ d)$ als $a\ b\ c\ d$
    schreiben.
    - Zusätzlich definieren wir, dass Abstraktionen stärker als Applikationen binden. Konkret heißt dies, dass der Ausdruck
    $λx.x\ y$ eine Abstraktion ist und nicht der Applikation von $λx.x$ auf $y$ entspricht. Um diese auszudrücken, setzen
    wir Klammern um die Abstraktion: $(λx.x)\ y$

#### 1.b) α-Reduktion und de Bruijn Notation
Wie bereits oben angedeutet kann es problematisch werden, wenn Variablen gebunden werden.

Im Beispiel $(λx.(λy.x\ y))\ y$
wird $(λx.(λy.x\ y))$ auf $y$ angewandt. Dafür ersetzten wir alle Vorkommnisse von $x$ in $(λy.x\ y)$ durch $y$ und
erhalten $(λy.y\ y)$.

Nun wird allerdings dasselbe Symbol für zwei unterschiedliche Zwecke benutzt. Das erste $y$
entspricht dem Argument, auf welches wir $(λx.(λy.x\ y))$ angewandt hatten, während das zweite $y$ kennzeichnet, wo das
Argument auf welches $(λy.y\ y)$ angewandt werden kann einzusetzen ist. Falls wir nun also diese $(λy.y\ y)$ auf $z$ anwenden wollen,
erwarten wir $y\ z$. Es wäre jedoch auch nachvollziehbar, falls wir die falsche Antwort $z\ z$ erhalten würden.

Um solche Fehler zu vermeiden ist es möglich Variablen umzunennen. Dies nennt sich α-Reduktion und verschiebt (im schlimmsten Fall)
die Missverständnisse auf die Ebene der Interpretation von λ-Termen durch ihre Autoren. Im Kontext des Lambda-Kalküls
sind also die Terme $(λx.(λy.x\ y))\ y$ und $(λx.(λz.x\ z))\ y$ sowie $(λx.(λy.x\ y))\ z$ (α-)equivalent.


Da α-Reduktion schwer zu automieren ist und sie neue Symbole einführt, ist es häufig, auch in unserer Implementation,
praktischer die de Bruijn Notation (de Bruijn 1972) zu verwenden. In dieser ersetzen wir Variablen durch Paare ganzer Zahlen,
von denen die Erste angibt, vor wie vielen Abstraktionen die Variable gebunden wurde, und die zweite kennzeichnet,
als wievielte sie von dieser Abstraktion gebunden wurde. Für freie (andernorts definierte) Variablen ist lediglich der
erste Wert relevant, welcher das Symbol encodiert.

Wir würden also $(λx.(λy.x\ y))\ y$ als $(λ.(λ.<1,1>\ <0,1>))\ <1,k>$ encodieren. Hier wurde aus dem freien $y$, $<1,k>$
welches wir, falls es nicht in einer Abstraktion vorkommt, als $y$ decodieren können.


#### 1.c) β-Reduktion und Call-By-Name Auswertung
Die zweite fundamentale Art von Reduktion von λ-Termen, welche für das Verständnis der Krivine-Maschine notwendig ist,
ist die β-Reduktion. β-Reduktion entspricht dem oben bereits verwendeten Konzept der Funktionsaufrufe.
Bei der β-Reduktion werden λ-Terme der Form $(λX.A)\ Y$ zu $A[X := Y]$, wobei $X$ die gebundenen Variablen, $Y$ die Argumente und $A$ der
Körper der Abstraktion sind.
$A[X := Y]$ entspricht dann dem λ-Term $A$, bei dem die Vorkommnisse von $X$ durch $Y$ ersetzt wurden.

Die Krivine-Maschine implementiert β-Reduktion im Sinne der Call-By-Name Auswertung, bei der die Argumente einer Funktion
erst ausgewertet werden, wenn diese in dem Körper der Funktion auftauchen. Es werden also unausgewertete λ-Terme in den Körper einer Abstraktion
substituiert bevor dieser ausgewertet wird.

#### 1.d)
weak head normal form

### 2. Parsen und Kompilieren von λ-Termen
Im Folgenden befindet sich der Scheme Code, der benutzt wird um λ-Terme für die Maschine vorzubereiten.

#### 2.a) Parser
Wir beginnen indem wir ein kleines Typsystem für den Lambda-Kalkül definieren. Hierbei behandeln wir jeden λ-Term als ein Paar. Für Applikationen ist diese Entscheidung selbsterklärend und auch bei der Abstraktion liegt es nah, diese in "Signatur" und Körper aufzuteilen. 
Für Variablen war die Entscheidung eine praktische, da diese beim Kompilieren durch die oben eingeführten Zahlenpaare in der de Bruijn Notation ersetzt werden.

In [ ]:
(define (attach-tag type-tag contents)
  (cons type-tag contents))

(define (type-tag typed-content)
  (car typed-content))

(define (content typed-content)
  (cdr typed-content))

(define (make-application f x)
  (attach-tag 'APP (cons f x)))

(define (make-abstraction binding body)
  (attach-tag 'ABS (cons binding body)))

(define (make-variable v k)
  (attach-tag 'VAR (cons v k)))

Das Parsen selbst besteht darin, dass eine Liste an Symbolen, wie sie nativ in/von LISP gelesen werden kann, in die oben beschriebenen Paare umgeformt wird. Das Endresultat ist also ein verschachtelter λ-Term, dessen Typ mit *car*, Inhalt mit *cdr*, erstes Element mit *cadr* und zweites Element mit *cddr* angefordert werden kann.

In [ ]:
;creates ast
(define (parse input)
  (cond ((and (list? input) (empty? (cdr input)))
         (parse (car input)))
        ((and (list? input) (not (empty? input)))
         (make-application (parse (car input)) (parse (cdr input))))
        ((symbol? input)
         (let ([stringput (symbol->string input)])
           (cond ((get-pos-of stringput #\λ)
                  (let ([splitted (split-abstraction stringput)])
                    (make-abstraction (car splitted) (parse (cdr splitted)))))
                 (else
                  (attach-tag 'VAR (string->symbol stringput))))))))

In [ ]:
;helper to find index of character in string
(define (get-pos-of string char)
  (let loop ((list (string->list string))
             (index 0))
    (cond ((empty? list) #f)
          ((equal? (car list) char) index)
          (else (loop (cdr list) (+ 1 index))))))

;creates pair of bindings and body for a given abstraction
(define (split-abstraction term)
  (let ([dot-index (get-pos-of term #\.)])
    (cons (string->symbol (substring term 0 dot-index)) (string->symbol (substring term (+ 1 dot-index))))))

#### 2.b) Kompilierer

In [ ]:
(define encode-var #hash(("a" . 1)
                         ("b" . 2)
                         ("c" . 3)
                         ("d" . 4)
                         ("e" . 5)
                         ("f" . 6)
                         ("g" . 7)
                         ("h" . 8)
                         ("i" . 9)
                         ("j" . 10)
                         ("k" . 11)
                         ("l" . 12)
                         ("m" . 13)
                         ("n" . 14)
                         ("o" . 15)
                         ("p" . 16)
                         ("q" . 17)
                         ("r" . 18)
                         ("s" . 19)
                         ("t" . 20)
                         ("u" . 21)
                         ("v" . 22)
                         ("w" . 23)
                         ("x" . 24)
                         ("y" . 25)
                         ("z" . 26)))

(define decode-var #hash((1 . "a")
                         (2 . "b")
                         (3 . "c")
                         (4 . "d")
                         (5 . "e")
                         (6 . "f")
                         (7 . "g")
                         (8 . "h")
                         (9 . "i")
                         (10 . "j")
                         (11 . "k")
                         (12 . "l")
                         (13 . "m")
                         (14 . "n")
                         (15 . "o")
                         (16 . "p")
                         (17 . "q")
                         (18 . "r")
                         (19 . "s")
                         (20 . "t")
                         (21 . "u")
                         (22 . "v")
                         (23 . "w")
                         (24 . "x")
                         (25 . "y")
                         (26 . "z")))

In [ ]:
;bindings are tracked as a list where every lambda prepends a list of the variables it bounded (in order of binding)
;TODO: returns #f if var isnt bound, otherwise it returns a pair of depth when bound (used to calculate v) and the how many-th argument the variable is (k)
(define (bound? var bound-list)
  (cond ((empty? bound-list)
         #f)
        ((index-of (caar bound-list) var)
         (cons (cdr (car bound-list)) (+ 1(index-of (caar bound-list) var))))
        (else (bound? var (cdr bound-list)))))

;adds variable/s being bound at depth to the bound-list
(define (bind binding depth bound-list)
  (cons (cons (binding-helper binding) depth) bound-list))

;creates list of variables (as symbols) that are bound by the first part of an abstraction
(define (binding-helper binding)
  (let ([listed (for/list ([chars (string->list (symbol->string binding))])
                  (string->symbol (string chars)))])
    (cdr listed)))

In [ ]:
;"compiles" first part of an abstraction
(define (lambda-computer term)
  (let ([stringput (symbol->string (car term))])
    (cons #\λ (- (string-length stringput) 1))))

(define (compile-abstraction term depth bound-list)
  (let ([new-bound-list (bind (car term) depth bound-list)])
  (make-abstraction (lambda-computer term) (compile (cdr term) (+ 1 depth) new-bound-list))))


In [ ]:
(define (compile-variable var depth bound-list)
  (let ([binding-info (bound? var bound-list)])
    (cond (binding-info
           (make-variable (- (- depth 1) (car binding-info)) (cdr binding-info)))
          ((not binding-info)
           (make-variable (+ (- depth 1) (hash-ref encode-var (symbol->string var))) 99999)))))


In [ ]:
(define (compile term depth bound-list)
  (cond ((eq? 'APP (car term))
         (make-application (compile (car (cdr term)) depth bound-list) (compile (cdr (cdr term)) depth bound-list)))
        ((eq? 'ABS (car term))
         (compile-abstraction (cdr term) depth bound-list))
        ((eq? 'VAR (car term))
         (compile-variable (cdr term) depth bound-list))))

### 3. Die Maschine
Die hier vorgestellte abstrakte Maschine in Scheme folgt der im original von Krivine postulierten Variante (Krivine 2007). In einem ersten Abschnitt werden die notwendigen Datenstrukturen vorgestellte, ehe auf das Verhalten der Maschiene mit ihrem Zustand und den Übergängen eingegangen wird.
#### 3.a) Datenstrukturen
Die Krivine-Machine benötigt drei Typen von Datenstrukturen: Closures, einen Stack und Environments. Im Folgenden wird vertieft auf diese drei Eingegangen. 
##### Eine Closure
Ein Closure ist ein Paar bestehend aus einem als Term bezeichneten ausführbaren λ-Ausdruck und einem Environment. Die Implementierung ist mit den notwendigen *getter* und *setter* Methoden versehen, wobei *'fst* und *'snd* das erste respektive das zweite Element eines Paars returnieren.

In [6]:
; closure data structure: a pair
(define (make-closure)
  (let ((c '()))
    (lambda (msg . args)
      (cond
        ((eq? msg 'set)
          (set! c (cons (car args) (cdr args))))
        ((eq? msg 'fst)
          (car c))
        ((eq? msg 'snd)
          (cadr c))
        ((eq? msg 'get) c)
        (else
          (display (string-append "\nERROR: Not supported command: " (symbol->string msg))) (newline))))))

##### Der Stack
Auf dem Stack befinden sich Closures. Implementiert wurde dieser in Scheme als Liste. Mittels den üblichen Befehlen *'push* und *'pop* können Closures auf den Stack geschoben oder vom Stack geholt werden. Ebenfalls enthalten ist die Methode *'display*. Damit ist es möglich die einzelnen Closures auf dem Stack in der Konsole auszugeben.

In [7]:
; stack data structure with push and pop
(define (make-stack)
  (let ((s '()) (n 0))
     (lambda (msg . args)  ; msg and arguments, if . used, args is seen as list which can be empty
       (cond
         ((eq? msg 'pop)
            (cond
              ((null? s)
                (display "\nERROR: Stack is empty\n"))
              (else
                (set! n (- n 1))
                (define tmpS s)
                (set! s (cdr s))
                (car tmpS)))) ;return the car element of stack
         ((eq? msg 'push)
            (set! n (+ n 1))
            (set! s (append (reverse args) s)))
         ((eq? msg 'get) s)
         ((eq? msg 'size) n)
         ((eq? msg 'display)
            (define clos (make-closure))
            (cond ((null? s)
                   (display '()))
                  (else
                   (let loop ((i 0))
                     (cond ((< i (length s))
                            (set! clos (list-ref s i))
                            (display "[") (display (clos 'fst)) (display ",") (display (clos 'snd)) (display "] ")
                            (loop (+ i 1))))))))
         (else
          (display (string-append "\nERROR: Not supported command: " (symbol->string msg))) (newline))))))

##### Ein Environment
Ein Environment wurde ebenfalls als Liste implementiert. Wenn ein Environment nicht leer ist, dann ist das erste Element selbst ein Environment. Alle folgenden sind Closures, welche mittels *'append* angefügt werden können. Die Datenstruktur bietet die Möglichkeit entweder das enthaltene Environment oder die Closure an Position *k* zrückzugeben.

In [8]:
; environment data structure: a list
(define (make-environment)
  (let ((e '()))
    (lambda (msg . args)
      (cond
        ((eq? msg 'append)    ; append arg at the end of list
          (set! e (append e args)))  ; append takes list, thus no (car args) needed
        ((eq? msg 'getHigh)
          (car e))
        ((eq? msg 'getK)
          (list-ref e (car args))) ; get the kth closure of this environment
        ((eq? msg 'get) e)        
        (else
          (display (string-append "\nERROR: Not supported command: " (symbol->string msg))) (newline))))))

#### 3.b) Zustand
Der Zustand der Krivine-Maschine ist das Tripel besthenden aus Term $T$, Stack $S$ und Environment $E$. Der Term ist der als nächstes auszuführende λ-Ausdruck. $E$ und $S$ sind zu Beginn leer.  
Im Unterschied zur ursprünglichen Idee von Krivine (2007, S. 201), wird in der hier präsentierten Implementierung nicht mit Zeigern gearbeitet. Dies ist der Tatsache geschuldet, dass die Sprache Scheme im Gegensatz etwa zur Sprache C keine Zeigervariablen kennt. Daher wird direkt mit den Objekten gearbeitet.  
Um einen Zustand festzuhalten, wurde die Datenstruktur *krivine-state* geschreiben, welche für alle drei Elemente des Zustands eine ensprechende *'get* und *'set* Methode anbietet.

In [9]:
; Krivine Machine state consisting of next to evaluate term T, stack S and environment E
(define (krivine-state)
  (let ((T '()) (S (make-stack)) (E (make-environment)))  ; on object creation elements are initialized
    (lambda (msg . args)  ; args is always a list, to get sinlge element use (car args)
       (cond
          ((eq? msg 'setT) (set! T (car args)))
          ((eq? msg 'setS) (set! S (car args))) 
          ((eq? msg 'setE) (set! E (car args))) 
          ((eq? msg 'getT) T)
          ((eq? msg 'getS) S)
          ((eq? msg 'getE) E)
          (else
             (display (string-append "\nERROR: Not supported command: " (symbol->string msg))) (newline))))))

#### 3.c) Zustandsveränderung
Krivine (2007, S. 201/204) beschreibt drei Zustandsveränderung in Abhängigkeit vom aussehen des Terms $T$, also des auszuführenden Ausdrucks. Im vorliegenden Fall wurden die Regeln derart implementiert, dass sie eine State-Objekt als Argument entgegen nehmen, die notwendigen Schritt ausführen und danach den aktualisierten Zustand zurückgeben. Es sind die die Applikation, die Abstraktion und das Ausführen einer Variable.  
Es gibt mehrere Situation in der die Krivine-Maschine nicht mehr weiter arbeiten kann und anhält (Krivine 2007, S.204). Diese Fälle werden bei der jeweiligen Regel kurz erläutert.  
##### Applikation
Eine Applikation liegt vor, wenn $T=(u)x$ ist. Der Ausdruck $x$ wird zusammen mit dem aktuellen Environment $E$ in einer Closure $\xi=<x,E>$ gespeichert und diese anschliessend auf den Stack gepushed. Für den nächsten Schritt wird $T=u$ gesetzt. Das Environment bleibt unverändert.  
Während dem Abarbeiten einer Applikation hält die Maschiene nie an.

In [21]:
; Application (λx.x)(λy.y)
(define (app s) 
  (define clos (make-closure))
  (clos 'set (cddr (s 'getT)) (s 'getE)) ; create closure with second part of T pair as new T and the current environment
  (define stack (s 'getS))
  (stack 'push clos)
  (s 'setS stack)  ; push it to stack
  (s 'setT (cadr (s 'getT))) ; T is the first element of the pair which is evaluated next
  s) ; resturn new state

##### Abstraktion
Eine Abstaktion liegt vor, wenn der Term mit einem $\lambda$ beginnt, also zum Beispiel $T=\lambda x_{1}\ldots\lambda x_{n}y$ ist und $y$ kein $\lambda$ vorangestellt hat. In diesem Fall wird ein neues Environment $E'$ erstellt. Das erste Element ist das bisherige $E$. Die weiteren Elemente sind die $n$ Closures, welche vom Stack geholt werden. Somit ist $E'=(E,\xi_1,\ldots,\xi_n)$. Für die Weiterverarbeitung ist $T=y$ und $E=E'$. Der Stack kann, muss aber nicht leer sein.  
Die Krivine-Maschine stoppt, falls die Zahl der vom Stack zu holenden Closures $n$ grösser ist als die Anzahl auf dem Stack befindliche Closures.

In [20]:
; Abstraction λx.x
(define (abs s)
  (define env (make-environment))
  (env 'append (s 'getE)) ; update E, the pointer to current environment
  (define stack (s 'getS))
  (cond ((> (cdr (cadr (s 'getT))) (stack 'size))
          (error "Too few Closures on Stack to perform Abstraction. Machine stopped" (stack 'get))))
  (let loop ((i (cdr (cadr (s 'getT))))) ; do number of pops
     (cond ((> i 0)
         (env 'append (stack 'pop))
         (loop (- i 1)))))
  (s 'setE env) ; this is new state environment
  (s 'setS stack) ; new stack
  (s 'setT (cddr (s 'getT))) ;T becomes the last element of former T pair
   s) ; return new state

##### Ausführen einer Variable
Wenn der Term nur eine einzelne Variable enthält, denn wird wiefolgt vorgegangen.  
Die Variable wurde im Kompilierschritt mit einem Wertepaar $<v,k>$ ersetzt. Dies gibt an in welchem Environment in welcher Closure die auszuführende Variable gebunden ist. Zuerst wird das richtige Enviroment gesucht. Ist $v=0$, dann handelt es sich um das aktuelle $E$. Falls $v>0$ wird das nächst höhere Environment $E_1$ gesucht, sprich das erste Element von $E$. Dies wird wiederholt, bis $E_v$ gefunden ist.  
Der Wert $k$ gibt an, welches Closure $\xi_k$ aus $E_v$ benötigt wird. Eine Closure besteht aus einem Term und einem Environment. $T$ wird nun zum Term von $\xi_k$ und $E$ zum Environment von $\xi_k$.  
Die Krivine-Maschine hält in diesem Verarbeitungsschritt aus zwei Gründen an. Ersten, wenn bei der Suche nach $E_v$ ein leeres Environment gefunden wird. Damit kann nicht weitergearbeitet werden. Zeitens stoppt die Maschine, wenn die Anzahl enthaltener Closures $n$ im Environment $E_v$ kleiner ist als $k$. Es kann keine Closure $\xi_k$ mit $k>n$ ausgwählt werden.

In [19]:
; Variable x
(define (var s)
  (define env (s 'getE))
  (define e (make-environment))
  (let ((v (cadr (s 'getT))) (k (cddr (s 'getT))))
    ;(display (string-append "v:" (number->string v) " k:" (number->string k))) (newline)
    ; get recursive the environment v and update state
    (let loop ((i 0))
     (cond ((< i v)
         (cond ((null? (env 'get))
             (error "Empty environment found. Machine stopped" (env 'get))))            
         (set! e (env 'getHigh))
         (set! env e)         
         (loop (+ i 1)))))
    ; get k closure in the updated state environment and update T, check if this is available
    (cond ((> k (length(env 'get)))
               (error "No Closure with needed index in environment. Machine stopped" (env 'get)))
          (else
               (let ((T '()) (E '()) (clos (make-closure)))
                   (set! clos (env 'getK k))
                   (set! T (clos 'fst))
                   (set! E (clos 'snd))
                   (s 'setT T) ; elements of clos are T and E
                   (s 'setE E))
               s)))) ; return new state

#### 3.d) Die Krivine-Routine
Mittels der vorgestellten Regeln kann nun die Krivine-Maschine mittels rekursivem Aufruf simuliert werden. Die Maschine ist fertig, wenn $S$ und $E$ erneut leer sind. Dann ist das Ergebnis der Reduktion in $T$ enthalten.  
Zum besseren Verständnis und zur Kontrolle zeigt die Krivine-Routine in jedem Schritt den Zustand der Maschine an. Das heisst Term, Stack und Environment werden ausgegeben. Aus Grunden der Einfachheit und zur besseren Übersicht werden die einzelenen Element eines Environments nicht detailliert dargestellt, sondern nur als Objekt angezeigt.

In [18]:
; Krivine Machine
(define (krivine-machine T)
  ; create state object
  (define state (krivine-state))
  (state 'setT T)
  (define n 0)
  ; eval loop
  (let eval ((state state) (n n))
     (cond ((= n 0)
            (display "Initial State:") (newline))
         (else
            (display (string-append "State after step " (number->string n) ":")) (newline)))
     (display "T: ") (display (state 'getT)) (newline)
     (display "S: ") ((state 'getS) 'display) (newline)
     (display "E: ") (display ((state 'getE) 'get)) (newline) (newline)
     (cond ((eq? (car (state 'getT)) 'APP)
              (eval (app state) (+ n 1)))
           ((and (not (null? ((state 'getS) 'get))) (eq? (car (state 'getT)) 'ABS))
               ; start abstraction only if stack is non empty
              (eval (abs state) (+ n 1)))
           ((eq? (car (state 'getT)) 'VAR)
              (eval (var state) (+ n 1)))
           ((and (null? ((state 'getS) 'get)) (null? ((state 'getE) 'get)))
              state)  ; termination condition
           ))
  (state 'getT))

Hier ist Test:

In [22]:
(krivine-machine '(APP (ABS (λ . 1) APP (VAR 0 . 1) VAR 0 . 1) ABS (λ . 1) VAR 0 . 1))

Initial State:
T: (APP (ABS (λ . 1) APP (VAR 0 . 1) VAR 0 . 1) ABS (λ . 1) VAR 0 . 1)
S: ()
E: ()

State after step 1:
T: (ABS (λ . 1) APP (VAR 0 . 1) VAR 0 . 1)
S: [(ABS (λ . 1) VAR 0 . 1),#<procedure>] 
E: ()

State after step 2:
T: (APP (VAR 0 . 1) VAR 0 . 1)
S: ()
E: (#<procedure> #<procedure>)

State after step 3:
T: (VAR 0 . 1)
S: [(VAR 0 . 1),#<procedure>] 
E: (#<procedure> #<procedure>)

State after step 4:
T: (ABS (λ . 1) VAR 0 . 1)
S: [(VAR 0 . 1),#<procedure>] 
E: ()

State after step 5:
T: (VAR 0 . 1)
S: ()
E: (#<procedure> #<procedure>)

State after step 6:
T: (VAR 0 . 1)
S: ()
E: (#<procedure> #<procedure>)

State after step 7:
T: (ABS (λ . 1) VAR 0 . 1)
S: ()
E: ()



'(ABS (λ . 1) VAR 0 . 1)

### Fazit

## Quellenverzeichnis
- Krivine, Jean-Louis (2007):   
   *A call-by-name lambda-calculus machine*, Springer, <https://link.springer.com/article/10.1007/s10990-007-9018-9> (letzter Zugriff 3.9.2021)
- Rojas, Raúl (2015):  
  *A Tutorial Introduction to the Lambda Calculus*, FU Berlin, <https://arxiv.org/abs/1503.09060> (letzter Zugriff 3.9.2021)
- De Bruijn, Nicolaas Govert (1972):  
  *Lambda calculus notation with nameless dummies, a tool for automatic formula manipulation, with application to the Church-Rosser theorem*, <https://doi.org/10.1016/1385-7258(72)90034-0> (letzter Zugriff 3.9.2021)